### 도쿄 부동산 사건 데이터 수집 
출처 :  <a href = 'http://www.oshimaland.co.jp'> 오시마랜드 </a>

사이트 정보 : 본 웹사이트는 자살사건, 시체유기사건, 살인사건 등 과거에 불미스러운 사건이 있었던 건물을 모아서 보여줍니다.

####도쿄 

```

```


In [ ]:
# !pip install googletrans==4.0.0-rc1
# !pip install webdriver-manager
# !pip install requests

## 에어비앤비 주요 지역 탐색 

#### 1. 영어로된 일본 지역 리스트 본어로 전처리 
- 오시마랜드는 영오도 지원되나 일본어로 검색하는게 정확도가 높다 판단하여 일본어로 변경해준다 
- googletrans를 이용해 번역할 경우 정확도가 떨어진다
- 45개 지역 하드코딩으로 번역

In [ ]:
from googletrans import Translator
import pandas as pd

main_district = pd.read_csv('C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Oshimaland_data\\jieun\\main_district_df2.csv')
# district_list  = main_district['neighbourhood_cleansed'].values
# translator = Translator()
# jp_district_list = list(map(lambda district: translator.translate(district, src='en', dest='ja').text.replace(' ',''),district_list))
# jp_district_list[: 10]


#### Inside AIrbne 주요 지역

In [ ]:
jp_district_list = [
    '新宿区', '台東区', '墨田区', '豊島区', '渋谷区',
    '港区', '世田谷区', '大田区', '中野区', '中央区',
    '北区', '葛飾区', '杉並区', '江東区', '江戸川区',
    '文京区', '荒川区', '板橋区', '品川区', '千代田区',
    '足立区', '目黒区', '練馬区', '府中市', '八王子市',
    '日野市', '国分寺市', '町田市', '青梅市', '調布市',
    '武蔵野市', 'あきる野市', '三鷹市', '小金井市', '東村山市',
    '多摩市', '国立市', '狛江市', '西東京市', '立川市',
    '小平市', '羽村市', '武蔵村山市', '奥多摩町', '昭島市',
    '福生市'
]

#### 2. geojosn 파일 이용
InsideAirbnb 제공 geo 파일 사용

In [ ]:
import json
import pandas as pd

In [ ]:
with open('C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Tokyo_Airbnb\\data\\neighbourhoods.geojson') as f:
    json_f = json.loads(f.read()) # json 라이브러리 이용

geo_df = pd.DataFrame(json_f)
geo_df = pd.read_json('C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Tokyo_Airbnb\\data\\neighbourhoods.geojson') # pd.read_json 이용


In [ ]:
geojson_path = 'C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Tokyo_Airbnb\\data\\neighbourhoods.geojson'
# gdf = gpd.read_file(geojson_path)
# gdf

with open(geojson_path) as f:
    geo_data = json.load(f)

geo_data

In [ ]:
features = geo_data['features']

geo_dict = {}
for feature in features:
    coordinates = feature["geometry"]['coordinates']
    neighbourhood = feature["properties"]['neighbourhood']
    if neighbourhood in list(main_district['neighbourhood_cleansed'].values):
          lat_lng = coordinates[0][len(coordinates[0])//2]
          geo_dict[neighbourhood] = lat_lng

geo_dict

## 크롤링 
- 크롤링 메서드 생성 

In [ ]:
# 좌표값 리스트를 한 줄 만들기 
flat_coordinates = [ring for sublist in geo_dict.values() for ring in sublist ]
print(flat_coordinates)

In [ ]:
import pandas as pd
import numpy as np 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import time

####  1. oshimaland 웹사이트 불러오기


In [ ]:
def loadWeb(driver, url):
    # header 추가하기 

    
    # 화면 최대 크기 설정 
    driver.get(url)
    driver.maximize_window()
    #모든 창이 로딩 될떄까지 기다려 줌 
    time.sleep(3)
    #팝업창 화면 전환 후 닫아주기
    multi_window = driver.window_handles
    if(len(multi_window)>1):
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        time.sleep(3)
        # 접근 페이지 다시 요청
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
    
    # 전체 지도 축소 
    # driver.execute_script("document.body.style.zoom='60%'")    
    # time.sleep(2)




#### 2. 부동산 관련 사건 정보 수집

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException

def get_fire(driver, tokyo_data_dict, district):
        # fire 객체 수집 
        fire_object_list = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div > div.mk-annotation-container')
        fire_list = fire_object_list.find_elements(By.CSS_SELECTOR, '.map-fire')
        print('fire 개수: ', len(fire_list))
        for fire in fire_list:
            try: 
                is_enabled = fire.is_enabled()
                actions = ActionChains(driver)                 
                if is_enabled:
                    # 클릭이 가능하며 현재 화면에 표시 될 때 클릭 수행
                    # print('click OK')
                    if fire.is_displayed():
                        # print('display OK')
                        # x_offset = 200  # x 좌표를 조정할 값
                        # y_offset = 200  # y 좌표를 조정할 값
                        # driver.execute_script("arguments[0].scrollIntoView();", fire)
                        # actions.move_to_element_with_offset(fire, x_offset, y_offset)
                        # actions.click()
                        # actions.perform()

                        # 팝업창 클릭                         
                        actions = ActionChains(driver)                 
                        actions.move_to_element(fire).click().perform()

                        # 팝업 정보 가져오기 
                        # time.sleep(5)
                        WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.property-info-date > span')))
                        info_date = driver.find_element(By.CSS_SELECTOR, 'li.property-info-date > span').text
                        info_address = driver.find_element(By.CSS_SELECTOR, 'li.property-info-address').text
                        info_content = driver.find_element(By.CSS_SELECTOR, 'div.popup-property-info > section > ul > li:nth-child(3)').text

                        tokyo_data_dict.append({
                            'district': district,
                            'date': info_date,
                            'address': info_address,
                            'content': info_content
                        })

                        time.sleep(3)

                    else:
                        pass
                        # print('display NOT')
                else:
                     pass
                    # print('click not')

            except (NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException) as e:
                # 에러 처리
                print(f"Error occurred: {e}")

                # 필요한 경우 WebDriver 인스턴스 재초기화 등의 작업 수행
                # driver = initialize_driver()

                continue


#### 3. 지역 검색 

3-1 도쿄 주요 지역 검색 

In [ ]:
from tqdm import tqdm, tqdm_notebook

tokyo_data_dict = []

def search_fire_distrinct(driver, jp_district_list):
    # 주요 지역 검색 
    for district in tqdm_notebook(jp_district_list):
        print(district)
        region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
        # 검색어 clear
        region_search_tab.clear()
        # 지역 검색
        region_search_tab.send_keys(district)
        time.sleep(2)
        # 검색 버튼 클릭 
        search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
        search_btn.send_keys(Keys.ENTER)
        # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
        time.sleep(8)
        # fire 객체 정보 수집 
        get_fire(driver,tokyo_data_dict, district)




3-2 도쿄 지역 위도 경도 검색

In [ ]:
tokyo_data_dict = []

def search_fire_coordinate(driver, geo_dict):
    # 주요 지역 위도 경도 검색 
    for key, value in tqdm_notebook(geo_dict.items()):
       
        for lat_lng in tqdm_notebook(value):
            print(key)
            region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
            region_search_tab.clear()  # 검색어 clear
            lng = lat_lng[1]
            lat = lat_lng[0]
            #경도 위도 검색
            search  = f'{lng} {lat}'
            region_search_tab.send_keys(search)
            time.sleep(2)
            # 검색 버튼 클릭 
            search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
            search_btn.send_keys(Keys.ENTER)
            # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
            time.sleep(8)
            # fire 객체 정보 수집 
            get_fire(driver,tokyo_data_dict, key)



### 크롤링 

In [ ]:
from selenium import webdriver
 
url = 'https://www.oshimaland.co.jp/'

# chrome 이용 
#  WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"}
# chrome_option = webdriver.ChromeOptions()
# driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_option)

# firefox 이용 
driver = webdriver.Firefox()
loadWeb(driver, url)

#### 발생 에러들

1. WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"} : Chrome driver 문제
2. StaleElementReferenceException: Message: The element with the reference b3212e1d-3be9-4d7a-9bd2-9f736fc42e4a is stale; either its node document is not the active document, or it is no longer connected to the DOM
3. AttributeError: 'NoneType' object has no attribute 'text'
4. MoveTargetOutOfBoundsException

In [1]:
# 주요 지역명 검색 
# search_fire(driver, jp_district_list) 
# 위도 경도 검색 
# 지도를 60% 정도로 축소 뒤 수집 
import warnings
warnings.filterwarnings("ignore")
dict_list = list(geo_dict.keys())[8:30]
new_dict = {key: value for key, value in geo_dict.items() if key not in dict_list}
search_fire_coordinate(driver, new_dict)

driver.quit()

NameError: name 'geo_dict' is not defined

In [ ]:
tokyo_data_dict

In [ ]:
driver.quit()

### 크롤링 데이터 저장

- 230725 Fussa Shi 지역 부터 데이터 수집 필요 ( 7개의 지역만 수집했음에도 2시간 이상 소요)

```
 'Fussa Shi', 'Bunkyo Ku', 'Shinjuku Ku', 'Adachi Ku', 'Sumida Ku', 'Nerima Ku', 'Taito Ku', 'Kodaira Shi', 'Koganei Shi', 'Higashimurayama Shi', 'Hino Shi', 'Kunitachi Shi', 'Kokubunji Shi', 'Itabashi Ku', 'Setagaya Ku', 'Ota Ku', 'Nakano Ku', 'Shibuya Ku', 'Toshima Ku', 'Chofu Shi', 'Suginami Ku', 'Tama Shi', 'Akishima Shi', 'Arakawa Ku', 'Machida Shi', 'Nishitokyo Shi', 'Meguro Ku', 'Fuchu Shi', 'Kita Ku', 'Tachikawa Shi', 'Hachioji Shi', 'Mitaka Shi', 'Okutama Machi', 'Musashino Shi', 'Musashimurayama Shi', 'Ome Shi', 'Akiruno Shi', 'Hamura Shi'
```

In [ ]:
 # Chuo Ku 지역까지 수집 완료 
geo_dict.keys()

In [ ]:
tokyo_df = pd.DataFrame(tokyo_data_dict)
tokyo_df = tokyo_df[tokyo_df['district']!='Fussa Shi']
tokyo_df

In [ ]:
# 결측치 제거 
df_cleaned = tokyo_df.dropna(how='any', axis=0)

for column in df_cleaned.columns:
    df_cleaned = df_cleaned[df_cleaned[column] != '']

# 중복데이터 제거 
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned

In [ ]:
df_cleaned.info()

In [ ]:
# tokyo_df.to_excel('selenium_tokyo_data_geo.xlsx')
df_cleaned.to_csv('selenium_tokyo_data_geo.csv')

In [ ]:
df = pd.read_csv('selenium_tokyo_data_geo.csv', index_col=0)

df

#### 기존 오시마 데이터랑 중복 확인 

In [ ]:
oshima_yun = pd.read_csv('../../Oshimaland_data/yunyoung/selenium_tokyo_data.csv', index_col=0)
oshima_ji = pd.read_csv('../../Oshimaland_data/jieun/oshimaland_dataset_final.csv')


In [ ]:
# 결측치 제거 
oshima_yun = oshima_yun.dropna(how='any', axis=0)

for column in oshima_yun.columns:
    oshima_yun = oshima_yun[oshima_yun[column] != '']

# 중복데이터 제거 
oshima_yun = oshima_yun.drop_duplicates()
oshima_yun = oshima_yun.reindex(columns=['district', 'address', 'content', 'date'])
oshima_yun.rename(columns={'content': 'info'}, inplace=True)
oshima_yun

In [ ]:
oshima_ji_rename = oshima_ji.reindex(columns=[ 'address', 'info', 'occurred_date'])
oshima_ji_rename.rename(columns={'occurred_date': 'date'}, inplace=True)
oshima_ji_rename = oshima_ji_rename.drop_duplicates()
oshima_ji_rename.reset_index(drop=True, inplace=True)
oshima_ji_rename.dropna(inplace=True)
oshima_ji_rename = oshima_ji_rename.reindex(columns = ['district', 'address', 'info', 'date'])
oshima_ji_rename['district'] = oshima_ji_rename['address'].apply(lambda x: next((district for district in jp_district_list if district in x), x))
oshima_ji_rename.reindex(columns=['district', 'address', 'info', 'occurred_date'])

oshima_ji_rename


In [ ]:
combined_df = pd.concat([oshima_yun, oshima_ji_rename], ignore_index=True)
combined_df = combined_df.drop_duplicates()
combined_df

In [ ]:
new_oshima = df.reindex(columns=['district', 'address', 'content', 'date'])
new_oshima.rename(columns={'content': 'info'}, inplace=True)
new_oshima

In [ ]:
combined_df2 = pd.concat([combined_df, new_oshima], ignore_index=True)
combined_df2 = combined_df2.drop_duplicates()
combined_df2

#### 중복 데이터 확인 
- 7개의 지역만 추가 트롤링 했을때 기존 807개의 데이터에서  1183개의 데이터 즉 376 개의 새로운 데이터 수집 
- 대략적으로 46개의 지역을 다 크롤링 할 시 2000개 이상의 데이터 추가 수집 가능 

In [ ]:
# address 컬럼의 값이 중복되면 하나만 남기고 제거
combined_df2 = combined_df2.drop_duplicates(subset=['address'], keep='first')
combined_df2

#### 지도 내 줌인 줌아웃 실패 

In [ ]:
# from selenium.webdriver.common.action_chains import ActionChains
# # https://stackoverflow.com/questions/64474653/how-to-webscrape-a-canvas-element-with-python-selenium
# canvas = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > canvas.rt-root')
# action = ActionChains(driver)

# map_canvas = driver.find_element_by_css_selector("div.mk-map-view > canvas.rt-root")

# from selenium.webdriver.common.action_chains import ActionChains

# # 지도 요소 위에서 마우스를 움직임
# actions = ActionChains(driver)
# actions.move_to_element(map_canvas).click().perform()

# actions.move_by_offset(0, 200).perform() # 줌인만 가능 

# #줌인 줌아웃 반복 
# action.move_to_element(canvas).move_by_offset(0, 0).click_and_hold().perform()
# action.move_by_offset(-200,0).release().perform()

#### Request API 이용한 방법 

In [ ]:
import json
from  bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import urllib.request as req
from tqdm import tqdm, tqdm_notebook
import time
import requests

In [ ]:
#TODO shinjuku ku test
search_region = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
search_region.send_keys(jp_district_list[20])
time.sleep(2)
# 검색 버튼 클릭 
search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
search_btn.send_keys(Keys.ENTER)

time.sleep(2)

In [ ]:
json_url = 'https://api.oshimaland.co.jp/map'
headers = {
    "User-Agent": "Mozilla/5.0",
    "Origin": "https://www.oshimaland.co.jp"
}
data = {
    # 'keys': ["1330021123012133", "1330021123012311", "1330021123013022", "1330021123013200"]
    'keys' : ['1330021123013021',  '1330021123013023',  '1330021123013201']
}

response = requests.post(json_url, headers=headers, json=data)
json_data = response.json()

In [ ]:
from itertools import chain

origin = list(data.values())
flatten_list = list(chain(*origin))
flatten_list

In [ ]:
jsonValueList = []
for idx in range(len(flatten_list)):
    for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
        jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])

In [ ]:
getDataList2 = []
for idx in tqdm_notebook(range(len(jsonValueList))):
    json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
    url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
    code = req.urlopen(url)
    soup = BeautifulSoup(code, 'html.parser')
    json_data = json.loads(soup.text)
    getDataList2.append({
        'info' : json_data['info'],
        'address' : json_data['ad'],
        'dt' : json_data['dt'],
        'cr' : json_data['cr'],
     })